<a href="https://colab.research.google.com/github/kirsveta14/Course/blob/learning/11_tidy_data_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
def get_data(path=""):
    """load covid-19 data"""
    base_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/"
    datasets = {
        "deaths": "time_series_covid19_deaths_global.csv",
        "confirmed": "time_series_covid19_confirmed_global.csv",
        "recovered": "time_series_covid19_recovered_global.csv",
    }
    for k, v in tqdm(datasets.items(), desc="load data"):
        df = pd.read_csv(base_url + v, sep=",")
        df.to_csv(os.path.join(path, v), sep=",", index=False)

DIR = ""
get_data(DIR)

load data: 100%|██████████| 3/3 [00:00<00:00,  4.42it/s]


# Материалы курса [Продвинутый анализ данных в Python](http://portal.moex.com/departments/hr/Training_and_development/internal_coach/Lists/List1/DispForm.aspx?ID=51)

## Краткий экскурс в Tidy Data"
### Cодержание:
- Что такое "Tidy Data"
- Примеры данных
- Примеры манипуляций с данными

#### Что такое "Tidy Data"?
Понятие "Tidy Data" или "Организованных данных" (TD, ОД) раскрывается в знаменитой статье Hadley Wickham [Tidy Data](http://vita.had.co.nz/papers/tidy-data.html)

> Подобно семьям, организованные наборы данных похожи друг на друга, а каждый неорганизованный набор данных беспорядочен по-своему. TD обеспечивает стандартизированный способ связывания структуры набора данных (его физического шаблона) с его сементикой (его значением).

Чтобы понять, что такое TD лучше рассмотреть это на примерах.

In [0]:
# Создадим 2 набора данных
df1 = pd.DataFrame({
    'treatmenta': [np.nan, 16, 3],
    'treatmentb': [2, 11, 1]
}, index=pd.Series(['John Smith', 'Jane Doe', 'Mary Johnson'], name='person')
)

df2 = df1.copy().T
df2.index = df2.index.rename('treatment')

In [34]:
df1

,treatmenta,treatmentb
person,,
John Smith,NaN,2
Jane Doe,16.0,11
Mary Johnson,3.0,1


In [5]:
df2

person,John Smith,Jane Doe,Mary Johnson
treatment,,,
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


#### Структура данных
Большинство статистических наборов данных представляют собой двумерные массивы (таблицы), состоящие из строк и столбцов. Столбцы почти всегда имеют названия, а строки иногда имеют названия.

__Что мы знаем о наших данных:__
- В __Датафрейме 1__ приведены некоторые данные о воображаемом эксперименте в формета, который очень часто встречается в "природе". В датафрейме 2 столбца и три строки, столбцы и строки помечены.
- Существует множество способов структурирования одних и тех же базовых данных.
- В __Датафрейме 2__ приведены те же данные, что и в __Датафрейме 1__, но строки и столбцы транспонированы.
- Данные в датафреймах одни и те же, но структура их разная
- Информации о строках и столбцах при этом недостаточно, чтобы описать, почему две таблицы представляют собой одни и те же данные.
- В дополнении к физической структуре нам нужно описать лежащую в основе данных __Семантику__ или значение, отображаемых в таблицах данных.

In [6]:
df1.equals(df2)

False

#### Семантика данных
- Любой набор данных представляет собой набор значений - __числа__ (если данные количественные) или __строки__ (если данные категориальные)
- Данные могут быть организованы двумя способами. Каждое значение может принадлежать к переменной и наблюдению.:
    - Переменные содержат все значения, которые измеряют одно и тоже наблюдение (например, высота, температура, продолжительность) для всех объектов
    - Наблюдение содержит все значения измерений для одного объекта (например, человек, день, гонка) для всех атрибутов
    
Для того, чтобы организовать данные экспериментов (__Датафрейм 1, Датафрейм 22__) нам необходимо разобраться в семантике данных данных:
- Человек - 3 уникальных имени
- Лечение - 2 возможных варианта (a, b)
- Результат - 5 или 6 возможных значений, в зависимости от того, как мы интерпретируем пустые значения
- Мы знаем, что тип лечения влияет на результат эксперимента и они взаимосвязаны
- В такой постановке задачи нашим наблюдением (объектом) будет человек-лечение-результат

Важно понимать - что от того как мы сами определили объект (наблюдение) зависит и вид наших данных.

### Tidy Data - это

Прежде чем двигаться дальше, давайте подведем предварительный итог и уложим концепцию __Tidy Data__ в 3 простых пункта:
1. Каждая __переменная__ формирует __столбец__
2. Каждое __наблюдение (объект)__ формирует __строку__ - нам важно понимать, что у нас является __наблюдением (объектом)__
3. Каждый __тип__ единиц наблюдения формирует __таблицу__

<img src="https://miro.medium.com/max/3840/1*7jjzhy4KknPz9hJVnC_w7w.png">

Чтобы делать преобразования неорганизованных данных в pandas есть все средства:
- 90% операций делаются с помощью функции __pd.melt__ (но метод работает только для одной катетегории, но об этом позднее)
- Для остальных операций необходим творческий подход

Чтобы идти дальше посмотрим как работает функция __pd.melt__

<img src="https://img1.daumcdn.net/thumb/R800x0/?scode=mtistory2&fname=https%3A%2F%2Ft1.daumcdn.net%2Fcfile%2Ftistory%2F25177F4E5863D58A0C">

In [9]:
# Создадим TD df3
df3 = pd.melt(
    df1.reset_index(),
    id_vars='person',
    var_name='treatment'
).assign(treatment=lambda x: x['treatment'].str.replace('treatment', ''))

df3

,person,treatment,value
0,John Smith,a,NaN
1,Jane Doe,a,16.0
2,Mary Johnson,a,3.0
3,John Smith,b,2.0
4,Jane Doe,b,11.0
5,Mary Johnson,b,1.0


Тех вводных, которые мы получили уже достаточно, чтобы понять, что не так с теми данными, которые мы загрузили в предыдущем ноутбуке - давайте загрузим один из датасетов и посмотрим подробнее

In [0]:
DIR = ""

In [0]:
deaths_stat = pd.read_csv(os.path.join(DIR, "time_series_covid19_deaths_global.csv"))

In [12]:
deaths_stat.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,21,23,25,30,30,30,33,36,36,40,42,43,47,50,57,58,60,64,68,72,85,90,95,104,106,109,115,120,122,127,132,136,153,168,169,173,178,187,193,205
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,23,24,25,26,26,26,26,26,26,27,27,27,27,28,28,30,30,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,313,326,336,348,364,367,375,384,392,402,407,415,419,425,432,437,444,450,453,459,463,465,470,476,483,488,494,502,507,515,522,529,536,542,548,555,561,568,575,582
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,29,31,33,33,35,35,36,37,37,37,37,40,40,40,40,41,42,42,43,44,45,45,46,46,47,47,48,48,48,48,49,49,49,51,51,51,51,51,51,51
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3


Думаю, на этом этапе уже понятно, что __каждое наблюдение формирует строку__ - в нашем же случае для каждой даты в датафрейме выделен один столбец.

In [0]:
deaths_stat_processed = pd.melt(
    deaths_stat,
    id_vars=deaths_stat.columns[:4],
    var_name="date",
    value_name="deaths"
)

In [14]:
deaths_stat_processed.head()

,Province/State,Country/Region,Lat,Long,date,deaths
0,NaN,Afghanistan,33.0000,65.0000,1/22/20,0
1,NaN,Albania,41.1533,20.1683,1/22/20,0
2,NaN,Algeria,28.0339,1.6596,1/22/20,0
3,NaN,Andorra,42.5063,1.5218,1/22/20,0
4,NaN,Angola,-11.2027,17.8739,1/22/20,0


Прежде, чем продолжить работать с этими данными нам остается только:
- Привести названия столбцов к удобному формату - никаких специалльных символов за исклюением нижнего подчеркивания, только нижний регистр, максимально понятно
- Привести столбцы к нужным типам - как это сделать корректно посмотрим далее:
    - Используем метод `df.info()` чтобы определить типы данных
    - Используем методы `df.astype(**)` чтобы преобразовать данные в нужный формат

In [0]:
# Переименовываем столбцы
deaths_stat_processed.columns = deaths_stat_processed.columns.str.lower().str.replace("/", "_")

In [16]:
deaths_stat_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32452 entries, 0 to 32451
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   province_state  9882 non-null   object 
 1   country_region  32452 non-null  object 
 2   lat             32452 non-null  float64
 3   long            32452 non-null  float64
 4   date            32452 non-null  object 
 5   deaths          32452 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 1.5+ MB


Что необходимо сделать:
1. Привести `date` к типу `date`
2. В идеале поработать с пропусками в поле `province_state` - пока оставляем без изменений

In [0]:
deaths_stat_processed["date"] = pd.to_datetime(deaths_stat_processed["date"], format="%m/%d/%y")

In [18]:
deaths_stat_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32452 entries, 0 to 32451
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   province_state  9882 non-null   object        
 1   country_region  32452 non-null  object        
 2   lat             32452 non-null  float64       
 3   long            32452 non-null  float64       
 4   date            32452 non-null  datetime64[ns]
 5   deaths          32452 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 1.5+ MB


In [19]:
deaths_stat_processed.head()

,province_state,country_region,lat,long,date,deaths
0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0
1,NaN,Albania,41.1533,20.1683,2020-01-22,0
2,NaN,Algeria,28.0339,1.6596,2020-01-22,0
3,NaN,Andorra,42.5063,1.5218,2020-01-22,0
4,NaN,Angola,-11.2027,17.8739,2020-01-22,0


Аналогичные преобразования мы сделаем с остальными файлами и сохраним их для дальнейшей работы

In [0]:
def process_data(path_from, path_to):
    """Process covid-19 data"""
    datasets = {
        "deaths": "time_series_covid19_deaths_global.csv",
        "confirmed": "time_series_covid19_confirmed_global.csv",
        "recovered": "time_series_covid19_recovered_global.csv",
    }
    for k, v in tqdm(datasets.items(), desc="process data"):
        df = pd.read_csv(os.path.join(path_from, v), sep=",")
        df_processed = pd.melt(
            df,
            id_vars=df.columns[:4],
            var_name="date",
            value_name=k
        )
        df_processed.columns = df_processed.columns.str.lower().str.replace("/", "_")
        df_processed["date"] = pd.to_datetime(df_processed["date"], format="%m/%d/%y")
        
        # Важно сохранять данные в сериализуемый формат, чтобы нам заново не пришлось работать с типами данных
        df_processed.to_pickle(
            os.path.join(path_to, "{0}.p".format(k)),
            compression="gzip"
        )

In [21]:
process_data(DIR, DIR)

process data: 100%|██████████| 3/3 [00:00<00:00,  5.96it/s]


Мы двигаемся дальше, а для вас домашнее задание для закрепления знаний о Tidy Data

# Домашнее задание

## Задание 1.1.0
- У нас есть справочник ages с информацией о дате ржодения каждого человека
- Создайте TD с использованием этих данных и df1 с учетом, что объект у нас остаётся тот же
- Результат запишите в переменную df4

In [68]:
df4 = df1.copy()
df4['ages'] = ['1/20/80', '11/25/89', '12/14/90']
df4['ages'] = pd.to_datetime(df4['ages'], format="%m/%d/%y")
df4 = pd.melt(
    df4.reset_index(),
    id_vars=['person', 'ages'],
    var_name='treatment'
).assign(treatment=lambda x: x['treatment'].str.replace('treatment', ''))
df4

,person,ages,treatment,value
0,John Smith,1980-01-20,a,NaN
1,Jane Doe,1989-11-25,a,16.0
2,Mary Johnson,1990-12-14,a,3.0
3,John Smith,1980-01-20,b,2.0
4,Jane Doe,1989-11-25,b,11.0
5,Mary Johnson,1990-12-14,b,1.0


## Задание 1.1.1
- создайте TD из df2
- результат запишите в переменную df5

In [48]:
df5 = pd.melt(
    df2.reset_index(),
    id_vars='treatment',
    var_name='person'
).assign(treatment=lambda x: x['treatment'].str.replace('treatment', ''))
df5

,treatment,person,value
0,a,John Smith,NaN
1,b,John Smith,2.0
2,a,Jane Doe,16.0
3,b,Jane Doe,11.0
4,a,Mary Johnson,3.0
5,b,Mary Johnson,1.0


## Задание 1.1.2
- Посчитайте в скольких экспериментах принял участие каждый человек
- Посчитайте средний возраст участников каждого из экспериментов
- Посчитайте средний результат каждого из экспериментов

In [89]:
df6 = pd.melt(
    df4,
    id_vars=['person'],
    value_vars='treatment'
)
df6.groupby('person').count()['value']

person
Jane Doe        2
John Smith      2
Mary Johnson    2
Name: value, dtype: int64

In [103]:
df7 = pd.melt(
    df4, 
    id_vars=['treatment', 'ages'],
    value_vars='person'
)
df7.groupby('treatment').

,ages,variable,value
treatment,,,
a,3,3,3
b,3,3,3
